In [13]:
import sys
import os
import glob
import random
import copy
from IPython.display import display, Markdown
sys.path.append('../..')

from model import processing
from matplotlib import pyplot as plt

from model import run
from model import plot_utils as pu
from model.amm.omnipool_amm import OmnipoolState, cash_out_omnipool, value_assets, usd_price, lrna_price
from model.amm.stableswap_amm import StableSwapPoolState
from model.amm.agents import Agent
from model.amm.trade_strategies import omnipool_arbitrage, invest_all, price_sensitive_trading
from model.amm.global_state import GlobalState, fluctuate_prices, historical_prices

# same seed, same parameters = same simulation result
random.seed(42)
# price_list = processing.import_binance_prices(['BTC', 'ETH', 'DOT'], start_date='Jan 1 2023', days = 120)

assets = {
    'USDA': {'usd price': 1, 'weight': 0.25},
    'USDB': {'usd price': 1, 'weight': 0.25},
    'USDC': {'usd price': 1, 'weight': 0.25},
    'USDD': {'usd price': 1, 'weight': 0.25},
}

assert sum([t['weight'] for t in assets.values()]) == 1

initial_tvl = 1000000

initial_state = StableSwapPoolState(
    tokens={
        tkn: initial_tvl * assets[tkn]['weight'] for tkn in assets
    },
    unique_id='stableswap',
    amplification=100,
    trade_fee=0.0
)

initial_agent = Agent(
    holdings={'USDA': 5000}
)


In [14]:
print(f"Agent adds liquidity: {initial_agent.holdings['USDA']} USDA. Trade fee = {initial_state.trade_fee}")
test_agent = initial_agent.copy()
test_pool = initial_state.copy().add_liquidity(
    agent=test_agent,
    tkn_add='USDA',
    quantity=initial_agent.holdings['USDA']
)
no_fee_agent = initial_agent.copy()
no_fee_state = initial_state.copy()
no_fee_state.trade_fee = 0
no_fee_state.add_liquidity(
    agent=no_fee_agent,
    tkn_add='USDA',
    quantity=initial_agent.holdings['USDA']
)
print(f"Agent receives: {test_agent.holdings['stableswap']} stableswap shares. Effective fee = {(1 - test_agent.holdings['stableswap'] / no_fee_agent.holdings['stableswap']) * 100}%")

Agent adds liquidity: 5000 USDA. Trade fee = 0.0
Agent receives: 4999.994208429242 stableswap shares. Effective fee = 0.0%


In [3]:
print(initial_state)

Stable Swap Pool: stableswap
********************************
trade fee: 0.0
shares: 100000.0
amplification constant: 100
tokens: (

    USDA
    quantity: 25000.0
    weight: 0.25

    USDB
    quantity: 25000.0
    weight: 0.25

    USDC
    quantity: 25000.0
    weight: 0.25

    USDD
    quantity: 25000.0
    weight: 0.25

)
error message:none


In [4]:
print (no_fee_agent.holdings['stableswap'])

4999.947008535732


In [5]:
initial_state.ann


25600

In [6]:
no_fee_state.shares

104999.94700853573

In [7]:
no_fee_agent

Agent: agent
********************************
trade strategy: None
holdings: (

    *USDA*: 0

    *stableswap*: 4999.9470085357
)

In [8]:
test_agent

Agent: agent
********************************
trade strategy: None
holdings: (

    *USDA*: 0

    *stableswap*: 4999.9470085357
)